# Scraping Specs from Jawa.gg

Jawa.gg is a popular online marketplace for selling pc parts and whole prebuilt pc's. I would like to scrape the specs of the pre built pc's sold. This will be broken down into parts. 

1. Access the webpage for sold pc's
2. Gather the links to each pc sold
3. Scrape the specs for each pc 
4. Repeat for all pages (up to 161 as of 1/1/2024)
5. Store in a sql database



In [1]:
# Setup
from bs4 import BeautifulSoup
import ssl
import urllib.request, urllib.parse
import pandas as pd
import time
import json

# note that when switching from work computer to laptop I got an error about ssl certificate verification failed
# this should bypass this for now, but wouldn't be used if I put this into production

import ssl


ssl._create_default_https_context = ssl._create_unverified_context


Search Results


In [2]:
# Read URL
sample = 'https://www.jawa.gg/shop/full-systems/gaming-pcs-show-sold~5c456b-7fa58?page=100'
html=urllib.request.urlopen(sample).read()
soup=BeautifulSoup(html,'html.parser')


At this point, I've inspected the html and found that each of the 20 pc's that are present on a page are under 

div class="tw-group tw-relative"

In [3]:
# Find all instances
s = soup.find_all('div', class_="tw-group tw-relative")
# Check the length (should be 20)
len(s)

20

In [4]:
# First item
pc = s[0]

In [5]:

# There seem to be two instances of a
# It turns out that the first instance is the real information, and the 
# second has to do with the button.
pc.find('a').get('href') # succesfully gets the link

'/product/12148/uranium-gtx-1070-10100f-16gb-ram-custom-gaming-pc'

In [6]:
# how about the title
pc.find('div',class_="tw-mt-1 tw-line-clamp-1 tw-overflow-ellipsis tw-text-xs md:tw-text-sm").get_text()

'"Uranium" GTX 1070 + 10100F, 16gb RAM Custom Gaming PC'

In [7]:
# The Price the pc was sold for is under this structure
pc.find('div',class_="tw-order-2 tw-text-sm tw-font-semibold tw-text-white").get_text()

'$629.99'

I would like to run two tests, first I would like to make sure that I can get all 20 pcs, then test on a different test site.

In [8]:
# Create an empty df to store data
small_df = pd.DataFrame(columns=['Title','Price','Link'],index=range(20))
# loop through the 20 pcs ( note that there might not be 20, so we use len)
for i in range(0,len(s)):
    current_pc = s[i] # select just the one of interest
    title = current_pc.find('div',class_="tw-mt-1 tw-line-clamp-1 tw-overflow-ellipsis tw-text-xs md:tw-text-sm").get_text()
    sale_price = current_pc.find('div',class_="tw-order-2 tw-text-sm tw-font-semibold tw-text-white").get_text()
    # Make sale price into an actual number
    sale_price = sale_price.replace('$','') # Remove the $
    sale_price = float(sale_price) # convert to a float since its a number with decimals
    link = current_pc.find('a').get('href')
    link = 'https://jawa.gg' + link # Add jawa.gg as prefix so the link works
    #print(title,'|-|',sale_price,'|-|',link)

    # Now, since i premade the dataframe, we can use iloc to fill the table
    # this is based on knowing the row and column where the data will go
    # The first argument is the row and the second is the column.
    small_df.iloc[i,0] = title
    small_df.iloc[i,1] = sale_price
    small_df.iloc[i,2] = link
small_df

,Title,Price,Link
0,"""Uranium"" GTX 1070 + 10100F, 16gb RAM Custom G...",629.99,https://jawa.gg/product/12148/uranium-gtx-1070...
1,🐢🍕Raphael🍕🐢 ||Gaming and Streaming PC-Nvidia G...,499.0,https://jawa.gg/product/12134/raphael-ororgami...
2,Snow Edition - Ryzen 7 5800X | ROG Strix 3080 ...,2399.95,https://jawa.gg/product/12120/snow-edition-ryz...
3,Entry Level White Gaming PC (Ryzen 5 2600 + RX...,550.0,https://jawa.gg/product/12129/entry-level-whit...
4,Starter DIY Set Up Just add case PSU and GPU,100.0,https://jawa.gg/product/10974/starter-diy-set-...
5,RTX 3080 // i9 12900KF // 32GB 5600Mhz DDR5 //...,2099.0,https://jawa.gg/product/12103/rtx-3080-i9-1290...
6,Gaming PC Intel i5 12400 RTX 3060 Ti,989.99,https://jawa.gg/product/12099/gaming-pc-intel-...
7,Intel I5 HTPC/Itx PC 16gb/500/Rx 6600,399.0,https://jawa.gg/product/11977/intel-i5-htpcitx...
8,Budget Gaming PC! Core i5 || 16 GB DDR4 || GTX...,347.0,https://jawa.gg/product/12091/budget-gaming-pc...
9,MAGENTA - Mid-Range Gaming PC - Ryzen 5 3700x ...,700.0,https://jawa.gg/product/12089/magenta-mid-rang...


Test 1 complete.

In [9]:
def basic_data_pull(link):
    html=urllib.request.urlopen(link).read()
    soup=BeautifulSoup(html,'html.parser')
    s = soup.find_all('div', class_="tw-group tw-relative")

    # Create an empty df to store data
    small_df = pd.DataFrame(columns=['Title','Price','Link'],index=range(20))
    # loop through the 20 pcs
    for i in range(0,len(s)):
        current_pc = s[i] # select just the one of interest
        title = current_pc.find('div',class_="tw-mt-1 tw-line-clamp-1 tw-overflow-ellipsis tw-text-xs md:tw-text-sm").get_text()
        sale_price = current_pc.find('div',class_="tw-order-2 tw-text-sm tw-font-semibold tw-text-white").get_text()
        # Make sale price into an actual number
        sale_price = sale_price.replace('$','') # Remove the $
        sale_price = float(sale_price) # convert to a float since its a number with decimals
        link = current_pc.find('a').get('href')
        link = 'https://jawa.gg' + link # Add jawa.gg as prefix so the link works
        #print(title,'|-|',sale_price,'|-|',link)

        # Now, since i premade the dataframe, we can use iloc to fill the table
        # this is based on knowing the row and column where the data will go
        # The first argument is the row and the second is the column.
        small_df.iloc[i,0] = title
        small_df.iloc[i,1] = sale_price
        small_df.iloc[i,2] = link
    return small_df

In [10]:
basic_data_pull('https://www.jawa.gg/shop/full-systems/gaming-pcs-show-sold~5c456b-7fa58?page=102')
# effective

,Title,Price,Link
0,Budget Gaming PC! Core i5 || GTX 1650 || 16GB ...,377.0,https://jawa.gg/product/11720/budget-gaming-pc...
1,Budget 1440p Gaming and Streaming PC,700.0,https://jawa.gg/product/11710/budget-1440p-gam...
2,"Mid-Range Used Custom PC, *RTX2060 I7-9700F*",650.0,https://jawa.gg/product/11709/mid-range-used-c...
3,SALE! Gaming/Streaming PC - Ryzen 7 5800x | RT...,1599.0,https://jawa.gg/product/11515/sale-gamingstrea...
4,SALE! Gaming/Streaming PC - Ryzen 5 5600 | 32g...,1199.0,https://jawa.gg/product/11517/sale-gamingstrea...
5,Budget Banger v3.0,400.0,https://jawa.gg/product/11655/budget-banger-v30
6,Custom Bundle for ToastyGaming,320.0,https://jawa.gg/product/11654/custom-bundle-fo...
7,"""Viper"" GTX 1070 Mid Range Gaming/Streaming PC",575.0,https://jawa.gg/product/11659/viper-gtx-1070-m...
8,☢️ 🟠Taskmaster☣️🔶- High Gaming PC-Intel i5 124...,995.0,https://jawa.gg/product/11651/taskmaster-high-...
9,HOLIDAY SALE: GTX 1660 Super + Ryzen 3600 + 16...,679.91,https://jawa.gg/product/11648/holiday-sale-gtx...


Great! I'm glad that worked, now I will use this function to get ALL the pc's.

Note there are 161 pages as of 1/1/2024

In [11]:
base = 'https://www.jawa.gg/shop/full-systems/gaming-pcs-show-sold~5c456b-7fa58?page='
big_df = pd.DataFrame(columns=['Title','Price','Link'])
for page in range(1,162):
    # combine the base with the page number
    page_link =  base+str(page)
    # Search on the current webpage and merge dataframes
    big_df = pd.concat([big_df,basic_data_pull(page_link)], ignore_index=True)
big_df= big_df.dropna(how='all')# Drop any extra rows
big_df.head()

,Title,Price,Link
0,🎋🌸⚔️👺☁️Nezuko||Gaming PC-AMD RX 7800 XT 16GB❄️...,1899.0,https://jawa.gg/product/26998/nezukoororgaming...
1,Ryzen 7 5700X | RTX 3070 | 32GB Ram | 1TB M.2 ...,1100.0,https://jawa.gg/product/26972/ryzen-7-5700x-or...
2,"AMD ""Anime"" Pink and White Theme CYBER MONDAY ...",725.0,https://jawa.gg/product/24994/amd-anime-pink-a...
3,ALL WHITE GAMING PC | RX 580 | I7 5930K EQUIVA...,499.99,https://jawa.gg/product/26931/all-white-gaming...
4,AMD RX 7600 | Ryzen 5 5500 | 1TB SSD | The Min...,879.0,https://jawa.gg/product/26741/amd-rx-7600-or-r...


In [12]:
big_df.describe()

,Title,Price,Link
count,3219,3219.0,3219
unique,3155,757.0,3219
top,Ryzen 5 3600 RX 6600 16GB RAM 1TB M.2 Wifi 5 B...,500.0,https://jawa.gg/product/26998/nezukoororgaming...
freq,4,84.0,1


Now to get the important details for a specficic product. 

Redo the previous loading process with this new link.

In [13]:
product = 'https://www.jawa.gg/product/26702/5600-x-6700xt-digital-dash-32gb-ram-wifi'
html=urllib.request.urlopen(product).read()
soup=BeautifulSoup(html,'html.parser')

One method to get the details are in this description meta content.

In [14]:
soup.find_all('meta', property="og:description")

[<meta content="CPU Brand: AMD, CPU Series: Ryzen 5, CPU Model: Ryzen 5 5600, CPU Socket: AM4, CPU Core count: 6 cores, GPU Chipset: AMD, GPU Brand: PowerColor, GPU Series: Radeon RX 6700 XT, GPU Memory: 12GB, Memory Capacity: 32GB, Memory Type: DDR4, Memory Form Factor: DIMM (Desktop), Internal Storage Capacity: 1TB, Internal Storage Interface: NVME SSD, Case Brand: Lian Li, Case Color: Black, Power Supply Wattage: 800W, Motherboard Brand: AMD, Motherboard Socket: AM4, CPU Cooler Type: Air, CPU Cooler Socket: AM4" property="og:description"/>]

Another is to turn this next data line into a dictionary.

In [15]:
data_dict = json.loads(soup.find('script',id="__NEXT_DATA__").get_text())
data_dict

{'props': {'pageProps': {'dehydratedState': {'mutations': [],
    'queries': [{'state': {'data': {'buyer_protection_policy': None,
        'category': {'id': 28, 'name': 'Gaming PCs'},
        'condition': 'used',
        'created_at': '2023-12-24T13:05:14.118Z',
        'description': 'AMD 5600\n\nAMD 6700XT\n\nMSI B550 Tomahawk\n\n32GB RAM 3200MHZ\n\nLian li 216\n\nDeepCool AK400 Digital\n\n1TB NVME\n\n850Watt EVGA Fully modular PSU\n\n\nGrown up build all about business \n\n\n6700XT>>>>3060TI',
        'height': None,
        'id': 26702,
        'images': {'ids': ['production/listings/gtowbjpydqzyorbdlfi2',
          'production/listings/vadrki0lcpgwyq6jhsft',
          'production/listings/ewaloqeptuvk2zzkkiit',
          'production/listings/wlczdbpkmkkzdsnww5gt',
          'production/listings/dx14obbzmpjav4fbawvk'],
         'source': 'cloudinary'},
        'is_insured': False,
        'is_on_sale': False,
        'is_private_listing': False,
        'is_published': True,
     

I will likely go the dictionary route. The next steps are to get the key pieces out of the dictionary, and make a new dataframe with the details. This will get added to big_df once completed.

Variables of interest: all specs, date listed, date sold, (create delta variable), original price, price, 

note that I only want the first state. 

In [16]:
# what does the data look like?
data_dict.items()

dict_items([('props', {'pageProps': {'dehydratedState': {'mutations': [], 'queries': [{'state': {'data': {'buyer_protection_policy': None, 'category': {'id': 28, 'name': 'Gaming PCs'}, 'condition': 'used', 'created_at': '2023-12-24T13:05:14.118Z', 'description': 'AMD 5600\n\nAMD 6700XT\n\nMSI B550 Tomahawk\n\n32GB RAM 3200MHZ\n\nLian li 216\n\nDeepCool AK400 Digital\n\n1TB NVME\n\n850Watt EVGA Fully modular PSU\n\n\nGrown up build all about business \n\n\n6700XT>>>>3060TI', 'height': None, 'id': 26702, 'images': {'ids': ['production/listings/gtowbjpydqzyorbdlfi2', 'production/listings/vadrki0lcpgwyq6jhsft', 'production/listings/ewaloqeptuvk2zzkkiit', 'production/listings/wlczdbpkmkkzdsnww5gt', 'production/listings/dx14obbzmpjav4fbawvk'], 'source': 'cloudinary'}, 'is_insured': False, 'is_on_sale': False, 'is_private_listing': False, 'is_published': True, 'is_sold_out': True, 'labels': [], 'last_published_at': '2023-12-24T13:05:14.112Z', 'last_sold_at': '2023-12-31T22:48:45.137Z', 'lengt

In [17]:
data_dict.keys()

dict_keys(['props', 'page', 'query', 'buildId', 'isFallback', 'isExperimentalCompile', 'gssp', 'scriptLoader'])

the data of interest is in props. Next level down is pageProps. Then dehydratedState. Then queries. There seem to be two queries for this page specifically, but the data I want is in the first, so i will take only the first. I will need to check later on a different link if this pattern is the same. Then the data is in state. Lastly, each variable is under data. 


I would like to use flow.io to make a chart illustrating this relationship.


In [18]:
raw = data_dict['props']['pageProps']['dehydratedState']['queries'][0]['state']['data']


supporting info

In [19]:
raw['name']
raw['description']
raw['created_at']
raw['original_price']
raw['price']


85000

actual specs. will require going down one more level.

In [20]:
specs=raw['specs']

In [21]:
specs['CPU Brand']
specs['CPU Series']
specs['CPU Model']
specs['CPU Socket']
specs['CPU Core count']
specs['GPU Chipset']
specs['GPU Brand']
specs['GPU Series']
specs['GPU Memory']
specs['Memory Capacity']
specs['Memory Type']
specs['Memory Form Factor']
specs['Internal Storage Capacity']
specs['Internal Storage Interface']
specs['Case Brand']
specs['Case Color']
specs['Power Supply Wattage']
specs['Motherboard Brand']
specs['Motherboard Socket']
specs['CPU Cooler Type']
specs['CPU Cooler Socket']

'AM4'

Does this pattern hold up for all of our observations? 

First step, combine into a new df.

In [26]:
details = pd.DataFrame(columns=['Name','Description','Created at','Original Price', 'Final Price',
                                'CPU Brand',
                                'CPU Series',
                                'CPU Model',
                                'CPU Socket',
                                'CPU Core count',
                                'GPU Chipset',
                                'GPU Brand',
                                'GPU Series',
                                'GPU Memory',
                                'Memory Capacity',
                                'Memory Type',
                                'Memory Form Factor',
                                'Internal Storage Capacity',
                                'Internal Storage Interface',
                                'Case Brand',
                                'Case Color',
                                'Power Supply Wattage',
                                'Motherboard Brand',
                                'Motherboard Socket',
                                'CPU Cooler Type',
                                'CPU Cooler Socket'],index=range(len(big_df['Link'])))

In [24]:
for k,href in enumerate(big_df['Link']):
    # open and read link
    html=urllib.request.urlopen(href).read()
    soup=BeautifulSoup(html,'html.parser')
    # Create dictionary
    data_dict = json.loads(soup.find('script',id="__NEXT_DATA__").get_text())
    # parse raw data and add to df
    raw = data_dict['props']['pageProps']['dehydratedState']['queries'][0]['state']['data']
    raw['name']
    raw['description']
    raw['created_at']
    raw['original_price']
    raw['price']
    
    specs=raw['specs']

    specs['CPU Brand']
    specs['CPU Series']
    specs['CPU Model']
    specs['CPU Socket']
    specs['CPU Core count']
    specs['GPU Chipset']
    specs['GPU Brand']
    specs['GPU Series']
    specs['GPU Memory']
    specs['Memory Capacity']
    specs['Memory Type']
    specs['Memory Form Factor']
    specs['Internal Storage Capacity']
    specs['Internal Storage Interface']
    specs['Case Brand']
    specs['Case Color']
    specs['Power Supply Wattage']
    specs['Motherboard Brand']
    specs['Motherboard Socket']
    specs['CPU Cooler Type']
    specs['CPU Cooler Socket']

https://jawa.gg/product/26998/nezukoororgaming-pc-amd-rx-7800-xt-16gbamd-ryzen-5-7600-6-core32gb-ddr51tb-nvme
https://jawa.gg/product/26972/ryzen-7-5700x-or-rtx-3070-or-32gb-ram-or-1tb-m2-nvme-ssd
https://jawa.gg/product/24994/amd-anime-pink-and-white-theme-cyber-monday-sale
https://jawa.gg/product/26931/all-white-gaming-pc-or-rx-580-or-i7-5930k-equivalent-or-16gb-ddr4-or-500gb-nvme
https://jawa.gg/product/26741/amd-rx-7600-or-ryzen-5-5500-or-1tb-ssd-or-the-mini-legend-v7-free-avatar-game-code-included
https://jawa.gg/product/26886/artic-beast-ryzen-7-5700x-32gb-2tb-rtx-4060
https://jawa.gg/product/26887/drk1-gaming-pc-custom-1060-windows-10-pro-intel-i5-desktop-800gb
https://jawa.gg/product/26876/dark-side-liquid-cooled-rig-intel-inte-14-core-32gb-ddr4-1tb-ssd-rtx-4060-ti
https://jawa.gg/product/26861/rtx-3060-ti-ryzen-5-3600-16gb-ram-1tb-nvme
https://jawa.gg/product/25309/buuamd-rx-5700-xt-8gb-amd-ryzen-5-3500x-6-core16gb-ddr4750gb-ssd
https://jawa.gg/product/23705/gaming-pc-core-i7-